<a href="https://colab.research.google.com/github/ARNAVKS/Maleria-Detection/blob/main/Malaria_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

cell-images-for-detecting-malaria.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
from zipfile import ZipFile
dataset = '/content/cell-images-for-detecting-malaria.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Extracted')

Extracted


In [4]:
import os

In [5]:
defected_files_=os.listdir('/content/cell_images/Parasitized')
normal_files_=os.listdir('/content/cell_images/Uninfected')

In [6]:
defected_files=[]
for files in defected_files_:
  if files.split('.')[1]!='db':
    defected_files.append(files)
normal_files=[]
for files in normal_files_:
  if files.split('.')[1]!='db':
    normal_files.append(files)

In [7]:
print(len(defected_files))
len(normal_files)

13779


13779

In [8]:
defected_files=defected_files[:4000]

In [9]:
normal_files=normal_files[:4000]

In [10]:
import cv2
t=cv2.imread('/content/cell_images/Parasitized/C101P62ThinF_IMG_20150918_151942_cell_68.png')

In [11]:
t.shape

(157, 148, 3)

In [12]:
defected_label=[1]*4000
normal_label=[0]*4000

In [13]:
label=defected_label+normal_label

In [14]:
len(label)

8000

In [15]:
data=[]
path='/content/cell_images/Parasitized/'
for img in defected_files:
  image=cv2.imread(path+img)
  image=cv2.resize(image,(128,128))
  data.append(image)
path='/content/cell_images/Uninfected/'
for img in normal_files:
  image=cv2.imread(path+img)
  image=cv2.resize(image,(128,128))
  data.append(image)

In [16]:
len(data)

8000

In [17]:
import numpy as np

In [18]:
X=np.array(data)
y=np.array(label)

In [19]:
X.shape

(8000, 128, 128, 3)

In [20]:
y.shape

(8000,)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.28,random_state=42)

In [23]:
X_train.shape,X_test.shape

((5760, 128, 128, 3), (2240, 128, 128, 3))

In [24]:
X_train_scaled,X_test_scaled=X_train/255,X_test/255

In [25]:
from tensorflow.keras import models, layers

In [26]:
model=models.Sequential()

In [27]:
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

In [29]:
model.fit(X_train_scaled,y_train,epochs=20,validation_split=0.1)

Epoch 1/20
162/162 [==============================] - 10s 24ms/step - loss: 0.6201 - accuracy: 0.6580 - val_loss: 0.3539 - val_accuracy: 0.8767
Epoch 2/20
162/162 [==============================] - 3s 19ms/step - loss: 0.2702 - accuracy: 0.9041 - val_loss: 0.1843 - val_accuracy: 0.9288
Epoch 3/20
162/162 [==============================] - 3s 18ms/step - loss: 0.1965 - accuracy: 0.9294 - val_loss: 0.1394 - val_accuracy: 0.9531
Epoch 4/20
162/162 [==============================] - 3s 18ms/step - loss: 0.1463 - accuracy: 0.9533 - val_loss: 0.1317 - val_accuracy: 0.9566
Epoch 5/20
162/162 [==============================] - 3s 18ms/step - loss: 0.1326 - accuracy: 0.9570 - val_loss: 0.1301 - val_accuracy: 0.9601
Epoch 6/20
162/162 [==============================] - 3s 20ms/step - loss: 0.0873 - accuracy: 0.9697 - val_loss: 0.1193 - val_accuracy: 0.9549
Epoch 7/20
162/162 [==============================] - 3s 18ms/step - loss: 0.0595 - accuracy: 0.9807 - val_loss: 0.1790 - val_accuracy: 0.947

In [30]:
model.evaluate(X_test_scaled,y_test)

70/70 [==============================] - 1s 7ms/step - loss: 0.4999 - accuracy: 0.9487


[0.49991869926452637, 0.9486607313156128]

In [31]:
y_train_pred=model.predict(X_train_scaled)
y_train_pred=[np.round(i) for i in y_train_pred]
y_test_pred=model.predict(X_test_scaled)
y_test_pred=[np.round(i) for i in y_test_pred]

70/70 [==============================] - 0s 7ms/step


In [32]:
from sklearn.metrics import accuracy_score

In [33]:
accuracy_score(y_train,y_train_pred)

0.9951388888888889

In [34]:
accuracy_score(y_test,y_test_pred)

0.9486607142857143